In [ ]:
import matplotlib.pyplot as pp
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tp

from functools import partial

In [ ]:
%config InlineBackend.figure_format = 'svg'

In [ ]:
pp.style.use('ggplot')

# Mixture

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/master/Chapter3_MCMC/data/mixture_data.csv',
                   header=None).astype('float32')[0]

In [ ]:
pp.figure(figsize=(12, 6))
pp.hist(data, bins=25);

In [ ]:
def unnormalized_log_probability(data, probability, locations, scales):
    rv_probability = tp.distributions.Uniform()
    rv_group = tp.distributions.Categorical(probs=[probability, 1.0 - probability])
    rv_locations = tp.distributions.Normal(loc=[120.0, 190.0], scale=[10.0, 10.0])
    rv_scales = tp.distributions.Uniform(low=[0.0, 0.0], high=[100.0, 100.0])
    rv_mixture = tp.distributions.MixtureSameFamily(
        mixture_distribution=rv_group,
        components_distribution=tp.distributions.Normal(
            loc=locations,
            scale=scales,
        ),
    )
    return (
        rv_probability.log_prob(probability) +
        rv_probability.log_prob(1.0 - probability) +
        tf.reduce_sum(rv_locations.log_prob(locations)) +
        tf.reduce_sum(rv_scales.log_prob(scales)) +
        tf.reduce_sum(rv_mixture.log_prob(data))
    )

state = [
    tf.constant(0.5),
    tf.constant([120.0, 190.0]),
    tf.constant([10.0, 10.0]),
]

bijector = [
    tp.bijectors.Identity(),
    tp.bijectors.Identity(),
    tp.bijectors.Identity(),
]

with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
    step_size = tf.get_variable(
        name='step_size',
        initializer=tf.constant(0.5, dtype=tf.float32),
        trainable=False,
        use_resource=True,
    )

kernel = tp.mcmc.TransformedTransitionKernel(
    inner_kernel=tp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=partial(unnormalized_log_probability, data),
        num_leapfrog_steps=2,
        step_size=step_size,
        step_size_update_fn=tp.mcmc.make_simple_step_size_update_policy(),
        state_gradients_are_stopped=True,
    ),
    bijector=bijector,
)

[
    posterior_probability,
    posterior_locations,
    posterior_scales,
], kernel = tp.mcmc.sample_chain(
    num_results=200000,
    num_burnin_steps=0,
    current_state=state,
    kernel=kernel,
)

In [ ]:
session = tf.Session()

session.run([
    tf.global_variables_initializer(),
    tf.local_variables_initializer(),
])

[
    posterior_probability_,
    posterior_locations_,
    posterior_scales_,
    kernel_,
] = session.run([
    posterior_probability,
    posterior_locations,
    posterior_scales,
    kernel,
])

print('Acceptance rate: {}'.format(kernel_.inner_results.is_accepted.mean()))

In [ ]:
pp.figure(figsize=(12, 12))

pp.subplot(311)
pp.plot(posterior_probability_)

pp.subplot(312)
pp.plot(posterior_locations_[:, 0])
pp.plot(posterior_locations_[:, 1])

pp.subplot(313)
pp.plot(posterior_scales_[:, 0])
pp.plot(posterior_scales_[:, 1]);